In [13]:
# importando funções a serem utilizadas
# obs: agora utiliza-se aqua
from qiskit import Aer
from qiskit.circuit.library import QFT
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.quantum_info import state_fidelity
# HHL e sistemas lineares
from qiskit.aqua.algorithms import HHL, NumPyLSsolver
from qiskit.aqua.components.eigs import EigsQPE
from qiskit.aqua.components.reciprocals import LookupRotation
from qiskit.aqua.operators import MatrixOperator
from qiskit.aqua.components.initial_states import Custom
import numpy as np

In [14]:
# função dos autovalores
def create_eigs(matrix, num_ancillae, num_time_slices, negative_evals):
    ne_qfts = [None, None]
    if negative_evals:
        num_ancillae += 1
        ne_qfts = [QFT(num_ancillae - 1), QFT(num_ancillae - 1).inverse()]

    return EigsQPE(MatrixOperator(matrix=matrix),
                   QFT(num_ancillae).inverse(),
                   num_time_slices=num_time_slices,
                   num_ancillae=num_ancillae,
                   expansion_mode='suzuki',
                   expansion_order=2,
                   evo_time=None,  # Valor do "tempo t". Pode ser setado em qualquer valor, exemplo: np.pi*3/4
                   negative_evals=negative_evals,
                   ne_qfts=ne_qfts)

In [15]:
# função da fidelidade
def fidelity(hhl, ref):
    solution_hhl_normed = hhl / np.linalg.norm(hhl)
    solution_ref_normed = ref / np.linalg.norm(ref)
    fidelity = state_fidelity(solution_hhl_normed, solution_ref_normed)
    print("Fidelidade:\t\t %f" % fidelity)

In [16]:
# Dados do problema ( função na forma: Ax + b = 0 )
# A = matrix
matrix = [[1, -1/2], [-1/2, 1]]
# b = vector
vector = [1, 0]

In [17]:
orig_size = len(vector)
matrix, vector, truncate_powerdim, truncate_hermitian = HHL.matrix_resize(matrix, vector)

# Inicialização do módulo dos autovalores
# create_eigs(matriz, número de ancilas, quantidade de passos, autovalores negativos)
eigs = create_eigs(matrix, 3, 50, False)
num_q, num_a = eigs.get_register_sizes()

# Inicialização do módulo de estado inicial
init_state = Custom(num_q, state_vector=vector)

# Inicialização do módulo de rotação
reci = LookupRotation(negative_evals=eigs._negative_evals, evo_time=eigs._evo_time)

# HHL
algo = HHL(matrix, vector, truncate_powerdim, truncate_hermitian, eigs,
           init_state, reci, num_q, num_a, orig_size)

In [18]:
#soluções finais
result = algo.run(QuantumInstance(Aer.get_backend('statevector_simulator')))
print("Solução:\t\t", np.round(result['solution'], 5))

result_ref = NumPyLSsolver(matrix, vector).run()
print("Solução Clássica:\t", np.round(result_ref['solution'], 5))

print("Probabilidade:\t\t %f" % result['probability_result'])
fidelity(result['solution'], result_ref['solution'])

Solução:		 [1.35867-0.j 0.71891-0.j]
Solução Clássica:	 [1.33333 0.66667]
Probabilidade:		 0.117814
Fidelidade:		 0.999470
